In [10]:
import numpy as np
import pandas as pd
#import seaborn as sns
from glob import glob
from PIL import Image
import matplotlib.pylab as plt
from sklearn.model_selection import train_test_split
from imutils import paths
import random

from subprocess import check_output

import keras
from keras.models import Sequential, load_model
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K
from keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from keras.utils import to_categorical
from keras.callbacks import ModelCheckpoint
from keras.applications.densenet import preprocess_input
from keras.preprocessing.image import (random_rotation, random_shift, random_shear, random_zoom)


In [2]:
# load a single image to np array
def get_image(img_path):
    img = load_img(img_path, target_size=(224, 224))#.convert('L')
    img = img_to_array(img)
    return img

In [16]:
# loading train images
train_dir = 'C:/Users/jdu12/Desktop/humpback/train/'
trainPaths = list(paths.list_images(train_dir))
print(len(trainPaths))
train = np.array([get_image(img_path) for img_path in trainPaths])
print(train.shape)

9082
(9082, 224, 224, 3)


In [3]:
def to_one_hot(labels):
    one_hot_labels = []
    id2class = dict()
    class2id = dict()      ##!!! class(integer) to id(whale class)
    counter = 0
    for id in labels:
        if id not in id2class.keys():
            one_hot_labels.append(counter)
            id2class[id] = counter
            class2id[counter] = id
            counter += 1
        else:
            one_hot_labels.append(id2class[id])
    one_hot_labels = to_categorical(one_hot_labels, num_classes = 4251)
    return one_hot_labels, class2id, id2class

In [4]:
train_label_dir = 'C:/Users/jdu12/Desktop/humpback/train.csv'
df = pd.read_csv(train_label_dir)

labels = df['Id']
labels,  , id2class = to_one_hot(labels)
#print(labels[0:10])

In [32]:
# split the data into train and val
# The dataset is super unbalanced, as there are many classes that contains only 1 image
# As a result, train/val data cannot be split before generating more data by augmentation
train_dir = 'C:/Users/jdu12/Desktop/humpback/train/'

num_categories = len(df['Id'].unique())
print(num_categories)
#validation = np.zeros((num_categories, 224, 224, 3))
validation_y = []

i = 0
for id in df['Id'].unique():
    validation_y.append(id2class[id])
    #im = df[df['Id'] == id].sample(1)
    #name =  np.array(im.get('Image'))[0]
    #im = get_image(train_dir + name)

    # https://www.kaggle.com/lextoumbourou/humpback-whale-id-data-and-aug-exploration
    x = random.randint(0, 3)
    if x == 0:
        print()
        #validation[i,:, :, :] = random_rotation(im, 30, row_axis=0, col_axis=1, channel_axis=2, fill_mode='nearest') * 255
    if x == 1:
        print()
        #validation[i,:, :, :] = random_shift(im, wrg=0.1, hrg=0.3, row_axis=0, col_axis=1, channel_axis=2, fill_mode='nearest') * 255
    if x == 2:
        print()
        #validation[i,:, :, :] = random_shear(im, intensity=0.4, row_axis=0, col_axis=1, channel_axis=2, fill_mode='nearest') * 255
    if x == 3:
        print()
        #validation[i,:, :, :] = random_zoom(im, zoom_range=(1.5, 0.7), row_axis=0, col_axis=1, channel_axis=2, fill_mode='nearest') * 255
    i = i + 1

4251





































































































































































































































































































































































































































































































































































































































































































































































































































































































































































































































In [33]:
print(len(validation_y))
print(to_one_hot(validation_y)[0])

4251
[[1. 0. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 1. 0. 0.]
 [0. 0. 0. ... 0. 1. 0.]
 [0. 0. 0. ... 0. 0. 1.]]


In [15]:
# define data generator
#use of an image generator for preprocessing and data augmentation
image_gen = ImageDataGenerator(
    #featurewise_center=True,
    #featurewise_std_normalization=True,
    rescale=1./255,
    #rotation_range=15,
    #width_shift_range=.15,
    #height_shift_range=.15,
    #horizontal_flip=True
)

In [17]:
# acquire the model
model = keras.applications.densenet.DenseNet121(include_top=True, weights=None, classes=4251)

In [24]:
batch_size = 32
epochs = 180

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model_dir = 'C:/Users/jdu12/Desktop/humpback/saved_model/DenseNet/'
routine_dir = model_dir + "routine-{epoch:02d}-{acc:.2f}-{val_acc:.2f}.hdf5"
routine_save = ModelCheckpoint(routine_dir, monitor='val_acc', verbose=0, save_best_only=False, save_weights_only=False, mode='auto', period=10)
best_dir = model_dir + "best-{epoch:02d}-{acc:.2f}-{val_acc:.2f}.hdf5"
best_save = ModelCheckpoint(best_dir, monitor='val_acc', verbose=0, save_best_only=True, save_weights_only=False, mode='auto', period=1)


model.fit_generator(image_gen.flow(train, labels, batch_size=batch_size),
                    steps_per_epoch=train.shape[0]//batch_size,
                    epochs=epochs, verbose=1, callbacks=[routine_save, best_save],
                    validation_data=image_gen.flow(validation, validation_y, batch_size), validation_steps=validation.shape[0]//batch_size)
          #class_weight=class_weight_dic)



ValueError: `x` (images tensor) and `y` (labels) should have the same length. Found: x.shape = (4251, 224, 224, 3), y.shape = (3,)

In [8]:
# loading test images
testPaths = list(paths.list_images('C:/Users/jdu12/Desktop/humpback/test/'))
print(len(testPaths))
test_images = np.array([get_image(img_path) for img_path in testPaths])/255
print(test_images.shape)

15610
(15610, 224, 224, 3)


In [58]:
# loading trained model
model = load_model('C:/Users/jdu12/Desktop/humpback/saved_model/DenseNet/' + 'routine-150-0.97.hdf5')

In [59]:
# predict!
pred = model.predict(test_images)

In [60]:
# testing and generating submission file
import warnings
from os.path import split

with open("sample_submission.csv","w") as f:
    f.write("Image,Id\n")
    top_5 = np.argsort(pred)[:,-1:-6:-1]   # get the top 5 most likely classes
    for i in range(top_5.shape[0]):
        cur_tags = ''
        cur_image_name = testPaths[i].split('/')[-1]
        for j in range(5):
            cur_tags = cur_tags + ' ' + class2id[top_5[i][j]]
        f.write("%s,%s\n" %(cur_image_name, cur_tags))     

In [52]:
top_5 = np.argsort(pred)[:,-1:-6:-1]
print(top_5.shape)
a = np.asarray([[1, 2, 3, 4, 5, 6, 7, 8],[1, 2, 3, 4, 5, 6, 7, 8]])
print(a[:,-1:-6:-1])


[[8 7 6 5 4]
 [8 7 6 5 4]]
